<a href="https://colab.research.google.com/github/Ricardomanuel1/Maestria_Ciencia_de_Datos/blob/main/III.%20Matematica_Ciencia_Datos/4_Clasificador_Bayesiano_Columna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':'170F7UczGqpHVc3Ed9nJIQPMHM9cZExua'})
downloaded.GetContentFile('iris.csv')

In [ ]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
random.seed(42) # Define la semilla

data = pd.read_csv('iris.csv', header=(0))

data = data.dropna(axis='rows') #removiendo NaN
# almacena los nombres de las classes
classes = np.array(pd.unique(data[data.columns[-1]]), dtype=str)

nrow, ncol = data.shape
print("Matriz de atributos: Número de filas:", nrow, " columnas: ", ncol)
attributes = list(data.columns)
data.head(10)

Matriz de atributos: Número de filas: 150  columnas:  5


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


In [ ]:
data = data.to_numpy()
nrow,ncol = data.shape
y = data[:,-1]
X = data[:,0:ncol-1]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [ ]:
print('Dados transformados:')
print('Media: ', np.mean(X, axis = 0))
print('Desviación típica:', np.std(X, axis = 0))

Dados transformados:
Media:  [-4.73695157e-16 -6.63173220e-16  3.31586610e-16 -2.84217094e-16]
Desviación típica: [1. 1. 1. 1.]


In [ ]:
from sklearn.model_selection import train_test_split
p = 0.8 # fracción de elementos en el conjunto de entrenamiento
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = p, random_state = 42)

In [ ]:
from scipy.stats import multivariate_normal

#matriz para almacenar probabilidades
P = pd.DataFrame(data=np.zeros((x_test.shape[0], len(classes))), columns = classes)

Pc = np.zeros(len(classes)) #fracción de elementos en cada clase
for i in np.arange(0, len(classes)):
    elements = tuple(np.where(y_train == classes[i]))
    Pc[i] = len(elements)/len(y_train)
    Z = x_train[elements,:][0]
    m = np.mean(Z, axis = 0)
    cv = np.cov(np.transpose(Z))
    for j in np.arange(0,x_test.shape[0]):
        x = x_test[j,:]
        pj = multivariate_normal.pdf(x, mean=m, cov=cv, allow_singular=True)
        P[classes[i]][j] = pj*Pc[i]

In [ ]:
P

,setosa,versicolor,virginica
0,5.222359e-81,2.945646e-04,9.166228e-06
1,3.993102e-03,7.431078e-31,8.110944e-69
2,1.067023e-248,8.603394e-17,1.815611e-07
3,2.820299e-80,1.521963e-02,3.042943e-05
4,4.932364e-95,3.479563e-03,1.695083e-06
5,3.420186e-03,5.383535e-24,1.726400e-59
6,2.005769e-46,1.157144e-03,6.897817e-11
7,1.744342e-143,1.474870e-14,5.570747e-05
8,3.431034e-90,1.163796e-05,1.720325e-05
9,1.142509e-53,1.670497e-02,9.673918e-09


In [ ]:
y_pred = []

for i in np.arange(0, x_test.shape[0]):
    c = np.argmax(np.array(P.iloc[[i]]))
    y_pred.append(classes[c])
y_pred = np.array(y_pred, dtype=str)
print(y_pred)

['versicolor' 'setosa' 'virginica' 'versicolor' 'versicolor' 'setosa'
 'versicolor' 'virginica' 'virginica' 'versicolor' 'virginica' 'setosa'
 'setosa' 'setosa' 'setosa' 'versicolor' 'virginica' 'versicolor'
 'versicolor' 'virginica' 'setosa' 'virginica' 'setosa' 'virginica'
 'virginica' 'virginica' 'virginica' 'virginica' 'setosa' 'setosa']


In [ ]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)

Accuracy: 0.9666666666666667
